In [ ]:
## Memorializing this as the easy way to lookup accounts


In [1]:
"""
Main Entry Point
"""
from pathlib import Path
from typing import List
import argparse
from datetime import datetime

import pandas as pd # type: ignore

import src.fetch_data # type: ignore
import src.core_transform # type: ignore
import cdutils.pkey_sqlite # type: ignore
import cdutils.hhnbr # type: ignore

# Current (doesn't really work without)
# data = src.fetch_data.fetch_data()


# Specific date
specified_date = datetime(2020, 12, 31)
data = src.fetch_data.fetch_data(specified_date)

# # # Core transformation pipeline
raw_data = src.core_transform.main_pipeline(data)

# Raw data with pkey appended
raw_data = cdutils.pkey_sqlite.add_pkey(raw_data)
raw_data = cdutils.pkey_sqlite.add_ownership_key(raw_data)
raw_data = cdutils.pkey_sqlite.add_address_key(raw_data)

In [2]:
househldacct = data['househldacct'].copy()
raw_data = cdutils.hhnbr.add_hh_nbr(raw_data, househldacct)

In [3]:
raw_data

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint
0,2020-12-31,150200005,MoneyGram Payment Systems Inc,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,4.464371e+08,0.0,0.0,4.464371e+08,0.0,41514.0,43261.0,44260.0,187340.0,2020-02-26 23:28:03
1,2020-12-31,61053112,City Of Taunton,Municipal Money Market,0.0,VAR,CK,CK18,ACT,0.001500,...,9.295986e+07,0.0,0.0,9.295986e+07,0.0,4183.0,33668.0,33967.0,97724.0,2020-02-26 23:28:03
2,2020-12-31,150544263,REDBROOK APARTMENTS LLC,Commercial Mortgages,42000000.0,VAR,CML,CM40,ACT,0.030000,...,4.200000e+07,0.0,0.0,4.200000e+07,42000000.0,57111.0,59661.0,61147.0,193906.0,2021-12-07 13:23:49
3,2020-12-31,60436522,Hodess Construction Corp,Investment Business Checking,0.0,VAR,CK,CK28,ACT,0.002000,...,2.027514e+07,0.0,0.0,2.027514e+07,0.0,3315.0,3942.0,4388.0,241524.0,2022-09-08 22:14:08
4,2020-12-31,6252261,Hutchens Holding II LLC,Commercial Swap Mortgage,13405500.0,VAR,CML,CM43,ACT,0.026586,...,1.614837e+07,0.0,0.0,1.614837e+07,13405500.0,3996.0,4209.0,4432.0,230789.0,2024-07-29 12:35:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91014,2020-12-31,150291616,"Fisher, Aquinnah C.",eChecking 16/17,100.0,None,CK,CK03,ACT,0.000000,...,-1.140470e+03,0.0,0.0,-1.140470e+03,100.0,NaN,NaN,NaN,NaN,NaT
91015,2020-12-31,150513086,"Brooks, Cadeem C.",Basic Checking,50.0,None,CK,CK02,ACT,0.000000,...,-1.331930e+03,0.0,0.0,-1.331930e+03,50.0,NaN,NaN,NaN,NaN,NaT
91016,2020-12-31,27039064,Bristol County Savings Bank,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,-5.259531e+04,0.0,0.0,-5.259531e+04,0.0,1221.0,1352.0,41577.0,255924.0,2022-09-27 22:46:27
91017,2020-12-31,150337783,Bristol County Savings Bank,Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,-6.343954e+04,0.0,0.0,-6.343954e+04,0.0,1221.0,1352.0,51094.0,255924.0,2022-09-27 22:46:27


In [4]:
import cdutils.loans.calculations

In [5]:
loan_category_df = cdutils.loans.calculations.categorize_loans(raw_data)

In [6]:
loan_category_df = loan_category_df[['acctnbr','Category']].copy()

In [7]:
df = pd.merge(raw_data, loan_category_df, on='acctnbr', how='left')

In [8]:
df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category
0,2020-12-31,150200005,MoneyGram Payment Systems Inc,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,0.0,0.0,4.464371e+08,0.0,41514.0,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN
1,2020-12-31,61053112,City Of Taunton,Municipal Money Market,0.0,VAR,CK,CK18,ACT,0.001500,...,0.0,0.0,9.295986e+07,0.0,4183.0,33668.0,33967.0,97724.0,2020-02-26 23:28:03,NaN
2,2020-12-31,150544263,REDBROOK APARTMENTS LLC,Commercial Mortgages,42000000.0,VAR,CML,CM40,ACT,0.030000,...,0.0,0.0,4.200000e+07,42000000.0,57111.0,59661.0,61147.0,193906.0,2021-12-07 13:23:49,CRE
3,2020-12-31,60436522,Hodess Construction Corp,Investment Business Checking,0.0,VAR,CK,CK28,ACT,0.002000,...,0.0,0.0,2.027514e+07,0.0,3315.0,3942.0,4388.0,241524.0,2022-09-08 22:14:08,NaN
4,2020-12-31,6252261,Hutchens Holding II LLC,Commercial Swap Mortgage,13405500.0,VAR,CML,CM43,ACT,0.026586,...,0.0,0.0,1.614837e+07,13405500.0,3996.0,4209.0,4432.0,230789.0,2024-07-29 12:35:49,CRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91014,2020-12-31,150291616,"Fisher, Aquinnah C.",eChecking 16/17,100.0,None,CK,CK03,ACT,0.000000,...,0.0,0.0,-1.140470e+03,100.0,NaN,NaN,NaN,NaN,NaT,NaN
91015,2020-12-31,150513086,"Brooks, Cadeem C.",Basic Checking,50.0,None,CK,CK02,ACT,0.000000,...,0.0,0.0,-1.331930e+03,50.0,NaN,NaN,NaN,NaN,NaT,NaN
91016,2020-12-31,27039064,Bristol County Savings Bank,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,0.0,0.0,-5.259531e+04,0.0,1221.0,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN
91017,2020-12-31,150337783,Bristol County Savings Bank,Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,0.0,0.0,-6.343954e+04,0.0,1221.0,1352.0,51094.0,255924.0,2022-09-27 22:46:27,NaN


In [9]:
import cdutils.inactive_date

df = cdutils.inactive_date.append_inactive_date(df)

In [10]:
df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category,inactivedate
0,2020-12-31,150200005,MoneyGram Payment Systems Inc,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,0.0,4.464371e+08,0.0,41514.0,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN,NaT
1,2020-12-31,61053112,City Of Taunton,Municipal Money Market,0.0,VAR,CK,CK18,ACT,0.001500,...,0.0,9.295986e+07,0.0,4183.0,33668.0,33967.0,97724.0,2020-02-26 23:28:03,NaN,NaT
2,2020-12-31,150544263,REDBROOK APARTMENTS LLC,Commercial Mortgages,42000000.0,VAR,CML,CM40,ACT,0.030000,...,0.0,4.200000e+07,42000000.0,57111.0,59661.0,61147.0,193906.0,2021-12-07 13:23:49,CRE,NaT
3,2020-12-31,60436522,Hodess Construction Corp,Investment Business Checking,0.0,VAR,CK,CK28,ACT,0.002000,...,0.0,2.027514e+07,0.0,3315.0,3942.0,4388.0,241524.0,2022-09-08 22:14:08,NaN,NaT
4,2020-12-31,6252261,Hutchens Holding II LLC,Commercial Swap Mortgage,13405500.0,VAR,CML,CM43,ACT,0.026586,...,0.0,1.614837e+07,13405500.0,3996.0,4209.0,4432.0,230789.0,2024-07-29 12:35:49,CRE,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91014,2020-12-31,150291616,"Fisher, Aquinnah C.",eChecking 16/17,100.0,None,CK,CK03,ACT,0.000000,...,0.0,-1.140470e+03,100.0,NaN,NaN,NaN,NaN,NaT,NaN,NaT
91015,2020-12-31,150513086,"Brooks, Cadeem C.",Basic Checking,50.0,None,CK,CK02,ACT,0.000000,...,0.0,-1.331930e+03,50.0,NaN,NaN,NaN,NaN,NaT,NaN,NaT
91016,2020-12-31,27039064,Bristol County Savings Bank,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,0.0,-5.259531e+04,0.0,1221.0,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN,NaT
91017,2020-12-31,150337783,Bristol County Savings Bank,Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,0.0,-6.343954e+04,0.0,1221.0,1352.0,51094.0,255924.0,2022-09-27 22:46:27,NaN,NaT


In [11]:
acct_df = df.copy()

In [12]:
acct_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91019 entries, 0 to 91018
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   effdate                 91019 non-null  datetime64[ns]
 1   acctnbr                 91019 non-null  object        
 2   ownersortname           91019 non-null  object        
 3   product                 91019 non-null  object        
 4   noteopenamt             91019 non-null  float64       
 5   ratetypcd               72634 non-null  object        
 6   mjaccttypcd             91019 non-null  object        
 7   currmiaccttypcd         91019 non-null  object        
 8   curracctstatcd          91019 non-null  object        
 9   noteintrate             91019 non-null  float64       
 10  bookbalance             91019 non-null  float64       
 11  notebal                 91019 non-null  float64       
 12  contractdate            91019 non-null  dateti

In [ ]:
pers = data['wh_pers'].copy()

In [ ]:
df.info()

In [ ]:
import src.additional_fields

additional_fields = src.additional_fields.fetch_data(specified_date)

In [ ]:
additional_fields_to_append = additional_fields['wh_acctcommon'].copy()

In [ ]:
import cdutils.input_cleansing

additional_fields_to_append_schema = {
    'acctnbr':'str'
}

additional_fields_to_append = cdutils.input_cleansing.enforce_schema(additional_fields_to_append, additional_fields_to_append_schema)

In [ ]:
df = pd.merge(df, additional_fields_to_append, how='left', on='acctnbr')

In [ ]:
df

In [ ]:
# OUTPUT_PATH = Path('acct_table.csv')
# df.to_csv(OUTPUT_PATH, index=False)

In [ ]:
# import hashlib

In [ ]:
# def mask_pii(data, columns_to_mask, length=10):
#     """
#     Create a masking layer

#     Pass in a dataframe to blackbox abstraction and get a dataframe returned with masked PII in specified fields

#     Parameters:
#     - data: raw data
#     - columns_to_mask: list of columns
#     - length: length of hash (10+ is recommended based on size of the data)
#     """
#     df_hashed = data.copy()

#     for col in columns_to_mask:
#         if col in df_hashed.columns:
#             df_hashed[col] = df_hashed[col].astype(str).apply(
#                 lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest()[:length]
#             )
#         else:
#             raise ValueError(f"Column {col} not found in dataframe passed in")
        
#     return df_hashed

In [ ]:
# columns_to_mask = [
#     'acctnbr',
#     'ownersortname',
#     'loanofficer',
#     'acctofficer',
#     'taxrptfororgnbr',
#     'taxrptforpersnbr'
#     ]

# masked_df = mask_pii(df, columns_to_mask)

In [ ]:
# masked_df

In [ ]:
# transposed_df = masked_df.head(3).T.reset_index()

In [ ]:
# transposed_df

In [ ]:
# OUTPUT_PATH = Path('masked_acct_table.csv')
# masked_df.to_csv(OUTPUT_PATH, index=False)